In [1]:
## Building a Chatbot

%pip install python-dotenv
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")
groq_api_key

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


'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'

In [2]:
## Bring Groq chatmodel 

%pip install langchain-groq
from langchain_groq import ChatGroq

model = ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000019733E76E60>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000019733EB0310>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="What is the capital of France?")]) 

AIMessage(content='The capital of France is **Paris**. \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 16, 'total_tokens': 28, 'completion_time': 0.021818182, 'prompt_time': 0.001905606, 'queue_time': 0.016494793, 'total_time': 0.023723788}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-559a3cf3-6841-4347-b7cf-b4d39d87f0b1-0', usage_metadata={'input_tokens': 16, 'output_tokens': 12, 'total_tokens': 28})

In [4]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="What is the capital of France?"),
        AIMessage(content="The capital of France is Paris."),
        HumanMessage(content="What is the capital of Germany? and Which is more beautiful for a trip?")
    ]
) 

AIMessage(content='The capital of Germany is Berlin. \n\nAs for which city is more beautiful, that\'s entirely subjective! Both Paris and Berlin have unique charms and beauty. \n\n**Paris** is often called the "City of Lights" and is renowned for its romantic atmosphere, iconic landmarks like the Eiffel Tower and Louvre Museum, charming cafes, and elegant architecture. \n\n**Berlin**, on the other hand, has a more edgy and alternative vibe. It\'s a city of history, with remnants of the Berlin Wall and numerous museums showcasing its tumultuous past. It also boasts vibrant street art, a thriving nightlife, and a diverse cultural scene.\n\nUltimately, the "more beautiful" city depends on your personal preferences. \n\n* **Do you prefer classic romance and elegance?** Paris might be your choice.\n* **Are you drawn to history, art, and a more modern, eclectic atmosphere?** Berlin could be a better fit.\n\n\nConsider what kind of experience you\'re looking for and what interests you most wh

In [5]:
## Message history and stateful sessions

%pip install langchain_community


from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model,get_session_history)

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
## session_id configuration

config = {"configurable":{"session_id":"Chat1"}}

In [7]:
response= with_message_history.invoke(
    [
        HumanMessage(content="What is the capital of France?"),
        AIMessage(content="The capital of France is Paris."),
        HumanMessage(content="What is the capital of Germany? and Which is more beautiful for a trip?")
    ],
    config=config
)

In [8]:
response.content

'The capital of Germany is Berlin. \n\nAh, the beauty question - that\'s subjective! Both Paris and Berlin are incredibly beautiful cities, each with their own unique charm. \n\n**Paris** is often called the "City of Lights" for its romantic ambiance, iconic landmarks like the Eiffel Tower and Louvre Museum, and charming cafes and streets. It\'s perfect for a classic European getaway with a focus on art, history, and romance.\n\n**Berlin** has a more edgy, historical vibe. It\'s a city of contrasts, with impressive architecture, vibrant nightlife, and a rich cultural scene. You\'ll find museums dedicated to its complex past, street art galore, and plenty of opportunities to explore its diverse neighborhoods.\n\nUltimately, the "more beautiful" city depends on your personal preferences. \n\n**Consider these factors:**\n\n* **What kind of atmosphere are you looking for?** Romantic and classic or edgy and historical?\n* **What are your interests?** Art, history, fashion, nightlife?\n* **W

In [9]:
with_message_history.invoke(
    [
        HumanMessage(content="where were we in last chat?")],
    config=config
)

AIMessage(content="We were discussing the capitals of France and Germany, and you asked which city was more beautiful for a trip. I gave you some insights into both Paris and Berlin, highlighting their unique charms.  \n\nIs there anything else you'd like to know about either city, or would you like to explore a different topic? 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 70, 'prompt_tokens': 318, 'total_tokens': 388, 'completion_time': 0.127272727, 'prompt_time': 0.012048082, 'queue_time': 0.018709996, 'total_time': 0.139320809}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-92cdfc36-ae79-4954-8727-dec972982bca-0', usage_metadata={'input_tokens': 318, 'output_tokens': 70, 'total_tokens': 388})

In [10]:
## Prompt Template

from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant.Answer all of my questions to best of your ability."),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt|model

In [11]:
chain.invoke({"messages":[HumanMessage(content="Does Islamabad known for its beauty?")]})

AIMessage(content="Islamabad is known for its **planned beauty**.  \n\nHere's why:\n\n* **Green spaces:**  It was designed with a lot of parks and green areas, giving it a lush and vibrant feel.\n* **Modern architecture:** Islamabad features a blend of modern and traditional architecture, creating a unique and visually appealing cityscape.\n* **Cleanliness:**  The city is generally considered to be quite clean and well-maintained.\n* **Mountain views:**  It's located in a valley surrounded by the Margalla Hills, providing stunning natural backdrops.\n\nWhile Islamabad may not be as historically rich or culturally diverse as some other Pakistani cities, its **planned design and emphasis on greenery** definitely contribute to its beauty.  \n\n\nDo you have any other questions about Islamabad? \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 166, 'prompt_tokens': 33, 'total_tokens': 199, 'completion_time': 0.301818182, 'prompt_time': 0.002341755, 'queue

In [12]:
with_message_history = RunnableWithMessageHistory(chain,get_session_history=get_session_history)

In [13]:
config2 = {"configurable":{"session_id":"Chat2"}}

response= with_message_history.invoke(
    [
        HumanMessage(content="What do you know about travel destinations?")],
    config=config2
)

In [14]:
response.content

"As a large language model, I have access to a vast amount of information about travel destinations! 🌎✈️\n\nTo give you the best answers, tell me what kind of information you're looking for. For example, are you interested in:\n\n* **Specific destinations?** (e.g., Paris, Tokyo, Machu Picchu)\n* **Types of destinations?** (e.g., beaches, mountains, historical sites)\n* **Travel planning tips?** (e.g., visa requirements, packing lists, budget advice)\n* **Cultural insights?** (e.g., local customs, cuisine, language)\n* **Travel inspiration?** (e.g., off-the-beaten-path destinations, unique experiences)\n\nThe more details you provide, the better I can tailor my response to your needs. 😄\n"

In [15]:
## Add more complexity in model 
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant translating messages.Answer all of my questions in {language}."),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt|model 

In [16]:
response=chain.invoke({"messages":[HumanMessage(content="Does Daarul Hakoomat mean capital of a state?")], "language":"Urdu"})
response.content
                      

'جی ہاں، "دَارُل حُکوْمَت" کا مطلب ایک ریاست کی راجگڑھ ہوتا ہے۔\n'

In [17]:
## wrap and chain all together

with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [18]:
config3 = {"configurable":{"session_id":"Chat3"}}

response= with_message_history.invoke(
    {"messages":[HumanMessage(content="Does Daarul Hakoomat mean capital of a state?")], "language":"Urdu"},
    config=config3
)
response.content

'جی ہاں، "دَارُ الحُکومت" ایک ریاست کی راجگھر یا دارالخلافت کے لیے استعمال ہوتا ہے۔ \n'